In [1]:
import sys
sys.path.append('../')

from envs.GridMaps import *
from utilities.transitionDatasetGeneration import *

In [5]:
#En los resultados de la memoria se presentan 2 distribuciones de casillas no demostradas para cada mapa. Para cada una de
#ellas se ha tenido que entrenar 1 ensemble. En el caso del grid de 5x5, se pueden encontrar los 2 ensembles en las 
#carpetas '5x5_5_model' y '5x5_5_modelv2' mientras que para el 14x14 en '5_model' y '5_modelv2'.

#Se recomienda no sobreescribir los modelos ya existentes en esas carpetas si se quieren replicar los resultados de la
#memoria, aunque también se pueden entrenar nuevos modelos para obtener resultados muy similares.


shape = "5x5"                     #Tamanho del grid '5x5' o '14x14'
store = False                     #'True' para guardar el ensemble generado en la carpeta

if shape == "5x5":
    maze = maze5x5["maze"]          #Nombre del mapa que se va a utilizar
    n_epochs = 15000                #Numero de epocas a entrenar cada modelo
    folder_name = "5x5_5_model"     #Nombre de la carpeta donde se guardan los modelos
    n_models = 30                    #Numero de modelos en el ensemble
    hidden_size = 32                #Numero de neuronas en las capas ocultas de los modelos

elif shape == "14x14":              
    maze = maze14x14["maze"]        #Nombre del mapa que se va a utilizar
    n_epochs = 120000               #Numero de epocas a entrenar cada modelo
    folder_name = "5_model"         #Nombre de la carpeta donde se guardan los modelos
    n_models = 5                    #Numero de modelos en el ensemble
    hidden_size = 512               #Numero de neuronas en las capas ocultas de los modelos
    
#Generacion del dataset offline
train_transitions, val_transitions = transition_generator(maze)

In [ ]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs, hidden_size, rand_seed=True)

    model.fit()
    models_arr.append(model)

In [7]:
if store: store_models(models_arr, folder_name)

In [ ]:
print('\nEVALUATION:\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []

for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([[5, 3, 3]]).to(device)
        inp2 = torch.FloatTensor([[5, 4, 3]]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0), '     ', np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.std(arr2_raw, axis=0), '     ', np.linalg.norm(np.std(arr2_raw, axis=0)))
